In [1]:
import pickle
import pandas as pd
import glob2
import plotly.express as px
import plotly.graph_objects as go
from influxdb import InfluxDBClient
from plotly.subplots import make_subplots
import log_loader_time

Welcome to JupyROOT 6.16/00


In [2]:
def extract_from_graal_and_save():
    dict_4_pd={
        "run":[],
        "subrun":[],
        "time":[],
        "Number_of_cluster_L1top_spX":[],
        "Number_of_cluster_L1bot_spX":[],
        "Number_of_cluster_L2top_spX":[],
        "Number_of_cluster_L2bot_spX":[],
        "Number_of_cluster_L1top_spV":[],
        "Number_of_cluster_L1bot_spV":[],
        "Number_of_cluster_L2top_spV":[],
        "Number_of_cluster_L2bot_spV":[],
        "Cluster_size_L1top_spX":[],
        "Cluster_size_L1bot_spX":[],
        "Cluster_size_L2top_spX":[],
        "Cluster_size_L2bot_spX":[],
        "Cluster_size_L1top_spV":[],
        "Cluster_size_L1bot_spV":[],
        "Cluster_size_L2top_spV":[],
        "Cluster_size_L2bot_spV":[],
        "Cluster_charge_L1top_spX":[],
        "Cluster_charge_L1bot_spX":[],
        "Cluster_charge_L2top_spX":[],
        "Cluster_charge_L2bot_spX":[],
        "Cluster_charge_L1top_spV":[],
        "Cluster_charge_L1bot_spV":[],
        "Cluster_charge_L2top_spV":[],
        "Cluster_charge_L2bot_spV":[]
    }
    time={}
    for filename, (run,)in glob2.iglob("/media/alb/Removibile/time_save/*", with_matches=True):
        with open (filename,"rb+") as savefile:
            start,end=pickle.load(savefile)
        time[int(run)]=start

        for filename, (subrun,)in glob2.iglob("/home/alb/GRAAL_out/srv_lab_mount/716392{}_*.txt".format(run), with_matches=True):
            with open (filename,'r') as graal_file:
                if subrun in time[int(run)].keys():
                    dict_4_pd["run"].append(int(run))
                    dict_4_pd["subrun"].append(int(subrun))
                    dict_4_pd["time"].append(time[int(run)][subrun])
                    for line in graal_file.readlines():
                        for key in dict_4_pd:
                            if "X" in key:
                                if key.split("_sp")[0].replace("_"," ") in line:
                                    dict_4_pd["{}_spX".format(key.split("_sp")[0])].append(float(line.split("-->")[1].split("|")[0].strip("\n").strip(" fC ")))
                                    dict_4_pd["{}_spV".format(key.split("_sp")[0])].append(float(line.split("-->")[2].strip("\n").strip(" fC ")))
                else:
                    print ("Missing, R{}, S{}".format(run,subrun))
        print ("RUN {} done".format(run))
        cluster_pd=pd.DataFrame(dict_4_pd)
        cluster_pd=cluster_pd.sort_values(by=["run","subrun"])
        # cluster_pd['time'] = pd.to_datetime(cluster_pd['time'],unit='s')
        # cluster_pd=cluster_pd.set_index("time")
        # cluster_pd["hour"]=cluster_pd.index.hour
        # cluster_pd["day"]=cluster_pd.index.date
        savefilename="/media/alb/Removibile/graal_extract/cluster_dataframe"
        with open (savefilename,"wb+") as savefile:
            pickle.dump(cluster_pd,savefile)  
    return cluster_pd

In [4]:
cluster_pd=extract_from_graal_and_save()

RUN 337 done
RUN 338 done
RUN 339 done
RUN 341 done
RUN 342 done
RUN 343 done
RUN 351 done
RUN 355 done
RUN 368 done
RUN 370 done
RUN 372 done
RUN 375 done
RUN 376 done
RUN 377 done
RUN 378 done
Missing, R387, S2
Missing, R387, S5
Missing, R387, S6
Missing, R387, S8
Missing, R387, S3
Missing, R387, S11
Missing, R387, S4
Missing, R387, S7
Missing, R387, S14
Missing, R387, S12
Missing, R387, S9
Missing, R387, S31
Missing, R387, S13
Missing, R387, S18
Missing, R387, S17
Missing, R387, S15
Missing, R387, S21
Missing, R387, S23
Missing, R387, S38
Missing, R387, S22
Missing, R387, S35
Missing, R387, S25
Missing, R387, S30
Missing, R387, S43
Missing, R387, S19
Missing, R387, S33
Missing, R387, S26
Missing, R387, S24
Missing, R387, S37
Missing, R387, S34
Missing, R387, S45
Missing, R387, S54
Missing, R387, S29
Missing, R387, S41
Missing, R387, S53
Missing, R387, S27
Missing, R387, S44
Missing, R387, S46
Missing, R387, S57
Missing, R387, S56
Missing, R387, S50
Missing, R387, S39
Missing, R387, 

In [27]:
cluster_pd['time'] = pd.to_datetime(cluster_pd['time'],unit='s')
cluster_pd=cluster_pd.set_index("time")
cluster_pd["hour"]=cluster_pd.index.hour
cluster_pd["day"]=cluster_pd.index.date

In [30]:
cluster_pd_1h=cluster_pd.resample("1h").mean()